In [ ]:
import itertools
import math
import numpy as np
import pandas as pd
from functools import reduce
from typing import Tuple, List, Optional, Generator

In [ ]:
def sign(n: int):
    return 1 if n > 0 else -1

def is_even(n: int):
    return n % 2 == 0

def is_odd(n: int):
    return n % 2 == 1

def product(ns: List[int]):
    return reduce(lambda x, y: x * y, ns, 1)

def gcd_recursion(a: int, b: int) -> int:
    """ Find GCD by recursing on a mod b """
    if(b == 0):
        return sign(a) * a

    return gcd_recursion(b, a % b)

def gcd_euclidian(a: int, b: int) -> int:
    """ Euler algo """

    # a = q * b - r
    q = a // b
    r = a - q * b
    
    # Stopping condition: r = 0
    if r == 0:
        return b
    
    # next iteration b = r * q_2 + r
    return gcd_euclidian(b, r)

def gcd(a: int, b: int, fn = None) -> int:
    """  """
    # ensure a >= b
    a, b = (a, b) if a >= b else (b, a)

    fn = fn or gcd_recursion
    return fn(a, b)

def extended_euclidian(a: int, b: int, state:Tuple[int, int, int] = None) -> int:
    """ Euler algo: find gcd(a, b) and x & y s.t. gcd(a, b) = x * a - y * b """

    # First time through, setup state, and ensure 'a' > 'b'
    state =  state or (((a, 1, 0), (b, 0, 1)) if a >= b else ((b, 1, 0), (a, 0, 1)))
   
    u = state[0]
    v = state[1]
    
    # Stopping condition: r = 0
    if v[0] <= 0:
        gcd = u[0]
        x = u[1] if a >= b else u[2]
        y = u[2] if a >= b else u[1]
        return (gcd, x, y)
    
    # a = q * b - r
    q = u[0] // v[0]
    w =  tuple(ui - q * vi for ui, vi in zip(u, v))
    state = (v, w)
    
    # next iteration b = r * q_2 + r
    return extended_euclidian(a, b, state)

def modulo_inverse(a, n):
    gcd, x, y = extended_euclidian(a, n)
    if gcd == 1:
        assert (a * x) % n == 1
        return x if x >= 0 else n + x 
    return np.NaN
    
def sieve_of_eratosthenes(n: int) -> Tuple[int,...]:
    """ Simple sieve to find primes up to n """
    n = n if n > 0 else -n
    if n < 2:
        return tuple()

    is_prime_sieve = np.full(n + 1, True)
    is_prime_sieve[0:2] = False
    is_prime_sieve[4::2] = False
    
    # Start with 2 and odd numbers from 3 to n
    sqrt_n = math.ceil(np.sqrt(n))
    for si in range(3, sqrt_n + 1, 2):
        if is_prime_sieve[si]:
            # Mark every multiple of si from si^2
            is_prime_sieve[si ** 2::si] = False
    return tuple(int(i) for i in np.flatnonzero(is_prime_sieve))

def primes_below(n: int) -> Tuple[int, ...]:
    """ """
    return sieve_of_eratosthenes(n - 1)

def next_prime(n: int):
    """ """
    for p in primes_below((math.trunc(np.sqrt(n)) + 1) ** 2):
        if p > n:
            return p
    return np.NaN

def is_prime(n: int) -> bool:
    """ check if is a prime by division: O(log(sqrt(n))) """
    for i in range(2, math.trunc(np.sqrt(n)) + 1):
        if n % i == 0:
            return  False
    return True

def ifactors(n: int) -> List[Tuple[int, int]]:
    """   """
    n_abs = sign(n) * n
    half_n = n_abs // 2
    is_prime = True
    factors = []
    for p in primes_below(half_n + 1):
        if p > half_n:
            break
        
        if n_abs % p == 0:
            is_prime = False
            k = 1
            m = n_abs // p
            while m % p == 0:
                m = m // p
                k += 1
            factors.append((p, k))
    
    if not factors:
        factors.append((n_abs, 1))
                
    return factors

def divisors(n: int) -> Tuple[int, ...]:
    """ """
    factors = ifactors(n)
    factor_primes = np.array([f[0] for f in factors] + [0,])
    # add one to every power, as we generate powers as (i mod factor_powers[j])
    factor_powers = np.array([f[1] for f in factors] + [0,]) + 1
    
    factors_count = len(factors)
    divisors_count = np.prod(factor_powers)

    # calc product of array of each prime factor to some power, varying from 0 to the max given from ifactors fn
    ds = sorted(int(np.prod([factor_primes[j] ** (i // np.prod(factor_powers[j - factors_count:]) % factor_powers[j])
                             for j in range(factors_count)]))
                for i in range(divisors_count))
    return tuple(ds)

def co_primes(n: int):
    """ """
    return set([a for a in range(1, n) if gcd(a, n) == 1])

def totient(n: int) -> int:
    """ Euler's phi function is the number of values less than a that are co-prime with n """
    return len(co_primes(n))

def order_of_powers(g: int, n: int) -> List[int]:
    """ g ^ k % n for k being co-prime with phi(n) """

    # order_of_powers = sorted(set([pow(g, k, n) for k in co_primes(totient(n))]))
    # keep all calcs mod n to remove overflow errors
    ks = co_primes(totient(n))
    order_of_powers = set()
    g_k = 1
    for k in range(1, n):
        g_k = g_k * g  % n
        if k in ks:
            order_of_powers.add(g_k)
    return sorted(order_of_powers)

def order(a: int, n: int):
    """ Multiplicative order of a mod n is the smallest k for which a^k mod n is 1 """
    if a > n or gcd(a, n) != 1:
        return np.NaN

    a_k = 1
    for k in range(1, n):
        a_k = a_k * a  % n
        if a_k == 1:
            return k
    
    return np.NaN

def is_order_n(a: int, n: int):
    """ Multiplicative order of a mod n is the smallest k for which a^k mod n is 1 """
    if a > n or gcd(a, n) != 1:
        return np.NaN
    
    ord_n = totient(n)
    # we can do better than all k < n by only looking at divisors of totient(n)
    phi_n_divisors = divisors(ord_n)
    for k in phi_n_divisors:
        if pow(a, k, n) == 1:
            return k == ord_n
    
    return np.NaN

def cyclic_group(a, n, op):
    group = set([pow(a, k, n) for k in range(1, n)])
    return group

def primitive_roots(n):
    # g is a primitive root modulo n if for every integer a coprime to n, there is some integer k for which gk ≡ a (mod n)
    
    # check n is form 2, 4, p^s, 2p^s, where s is any positive integer and p is an odd prime
    factors = ifactors(n)
    if any((len(factors) < 1 or 2 < len(factors),  
            (len(factors) == 2 and (factors[0][0] != 2 or factors[0][1] > 1)),
            (len(factors) == 1 and factors[0][0] == 2 and factors[0][1] > 2),
            (len(factors) == 1 and factors[0][0] < 2))):
        return [] # Exception("No primitive roots exist")
    
    # find smallest  root
    ord_n = totient(n)
    g = None
    for a in co_primes(n):
        if order(a, n) == ord_n:
            g = a
            break
    
    # There are phi(phi(n)) roots: return all roots using factors co-prime with phi(n)
    prime_roots = order_of_powers(g, n)
     
    assert len(prime_roots) == totient(ord_n)
    assert all(pow(g, ord_n, n) == 1 for g in prime_roots)
    return prime_roots

# Assignment 3

Stuart

a = 2353
b = 20837
c = 500551
d = 29213

Q6 = iii)

__Question 1 (10 marks):__

For the number $a$ given to you on page 1 answer the following:

* Show that a can be written as the sum of squares in two different ways. 
    
* Hence apply Euler’s method to factorise a.

In [ ]:
def diff_of_squares(n: int) -> Generator[Tuple[int, int], int, None]:
    """ Fermat factorisation: find two integers that when squared, the difference is n """
    sqrt_n = int(math.sqrt(n))    
    v, v2 = 1, 1
    squares = {v2: v}
    for u in range(sqrt_n, n // 2 + 1):
        u2 = u ** 2
        u2_minus_n = u2 - n
        while v2 < u2_minus_n:
            v += 1
            v2 = v ** 2
            squares[v2] = v
        squares[u2] = u
        if u2_minus_n in squares:
            yield (u, squares[u2_minus_n])
    return

def sum_of_squares(n: int) -> Generator[Tuple[int, int], int, None]:
    """ Find two integers that when squared, add up to n """
    sqrt_n = int(math.sqrt(n))
    u, u2 = 1, 1
    squares = {u2: u}
    for v in range(sqrt_n, sqrt_n // 2 + 1, -1):
        n_minus_v2 = n - v ** 2
        while u2 < n_minus_v2:
            u += 1
            u2 = u ** 2
            squares[u2] = u
        if n_minus_v2 in squares:
            yield (squares[n_minus_v2], v)
    return

## Fermat factorisation

Find a difference of squares such that $n = u^2 - v^2$.

Then factorise as $n = (u - v)(u + v)$ given that $u^2 - v^2 = (u - v)(u + v)$.

## Euler factorisation

Find two sum of squares such that $n = a^2 + b^ 2 = c^2 + d^2$.

Rearrange to: $a^2 - c^2 = d^2 - b^2 => (a - c)(a + c) = (d - b)(d + b)$  $(1)$.

$n$ is odd (otherwise divide out factors of 2), so one term in both $a^2 + b^2$ and $c^2 + d^2$ is odd and the other term even.  Assume that $a$ and $c$ are the even terms.

$k = gcd(a-c, b-d)$ so $a - c = kl$, $b - d = km$ and $gcd(l, m) = 1$ (otherwise a common factor would increase $gcd(a -c, b - d)$ and k must be even as $a - c$ and $b - d$ are even.

$h = gcd(a + c, b + d)$ giving $a + c = hm'$, $b + d = hl'$ and $gcd(l', m') = 1$ and h must also be even.

Substituting into (1) gives $klhm' = kmhl'$ so $lm' = ml'$ but since $gcd(l, m) = 1 = gcd(l', m')$, this is only possible if $l = l'$ and $m = m'$.

Hence $(a + c) = hm$ and $(d + b) = hl$.

Brahmagupta Identity: $(u^2 + v^2)(w^2 + x^2) = (uw - vx)^2 + (ux + vw)^2 = (uw + vx)^2 + (ux -vw)^2$.

Hence $(k^2 + h^2) (l^2 + m^2) = (kl + hm)^2 + (km - hl)^2  = ((a-c)+(a+c))^2 + ((b-d)+(b+d))^2 = (2a)^2 + (2b)^2  = 4a^2 + 4b^2 = 4(a^2 + b^2) = 4n$

As $k$ and $h$ are both even, $(k^2 + h^2) = 4((\frac{k}{2})^2 + (\frac{h}{2})^2)$.

Giving: $n = ((\frac{k}{2})^2 + (\frac{h}{2})^2) (l^2 + m^2)$.



In [ ]:
a = 2353
print(f"a = {a}")

def euler_factorisation(u_vs):
    print("\nEuler Factorisation")
    for u, v in u_vs:
        print(((u, v), f"{u ** 2} + {v ** 2} = {u ** 2 + v ** 2}"))
    a, b = u_vs[0] if is_even(u_vs[0][0]) else (u_vs[0][1], u_vs[0][0])
    c, d = u_vs[1] if is_even(u_vs[1][0]) else (u_vs[1][1], u_vs[1][0])
    k = gcd(a - c, d - b)
    h = gcd(a + c, d + b)
    l = (a - c) // k
    m = (d - b) // k
    assert (a + c) // h == m and (b + d) // h == l
    print(f"((k/2)^2 + (h/2)^2)(l^2 + m^2) = ({int(k/2)**2} + {int(h/2)**2})({l**2} + {m**2}) = {int(k/2)**2 + int(h/2)**2}.{l**2 + m**2} = {((int(k/2)**2 + int(h/2)**2) * (l**2 + m**2))}")
    return (int(k/2) ** 2 + int(h/2) ** 2, l ** 2 + m ** 2)

u_vs = tuple(u_v for u_v in sum_of_squares(a))
print(u_vs)
if (len(u_vs) >= 2):
    euler_factorisation(u_vs[0:2])

print("\nFermat factorisation")
for u, v in itertools.islice(diff_of_squares(a), 2):
    print(((u, v), f"{u ** 2} - {v ** 2} = {u ** 2 - v ** 2}"))
    print(f"u^2 - v^2 = (u - v)(u + v) => {u ** 2} - {v ** 2} = ({u} - {v})({u} + {v}) = {u - v}.{u + v} = {(u - v) * (u + v)}")

__Question 2 (10 marks):__

Take the number $b$ assigned to you on page 1 and apply the gcd method to find its smallest factor.
    
What is the value of $P_0$ that you need to guarantee finding a factor given that $b$ is composite?

In [ ]:
b = 20837
# print(ifactors(b))

print("\nEuclid’s Algorithm  to find factors")
print(f"  using primes up to the sqrt({b}) = {int(np.sqrt(b))}")
p0 = 1
prime_generator = primes_below(int(np.sqrt(b))+1)
for p in prime_generator:
    p0 *= p
print(f"  ... gives P0 = {p0}")
u = gcd(p0, b)
v = b // u
print(f"{u}.{v} = {u * v}")


__Question 3 (10 marks):__

Take the number $c$ assigned to you on page and use the $p-1$ method to find one factor of $c$. You may assume that $c-1$ factorises into primes of size $< 100$.

### Pollard p-1 algorithm

Let $n$ be a composite with prime factor $p$. By __FLT__ for all integers $a$ coprime to $p$ and for all positive integers $K$: $a^{{K(p-1)}} \equiv 1{\pmod {p}}$.

If a number x is congruent to 1 modulo a factor of n, then the gcd(x − 1, n) will be divisible by that factor.

The idea is to make the exponent a large multiple of $p − 1$ by making it a number with very many prime factors; generally, we take the product of all prime powers less than some limit $B$. Start with a random $x$, and repeatedly replace it by ${\displaystyle x^{w}{\bmod {n}}}$ as $w$ runs through those prime powers. Check at each stage, or once at the end if you prefer, whether $gcd(x − 1, n)$ is not equal to 1.


In [ ]:
def prime_powers_to(limit: int) -> List[int]:
    """ """
    # Extend b_primes up to the b$
    prime_powers = []
    powers_keys = []
    for p in sieve_of_eratosthenes(limit):
        if p > limit:
            break
        prime_power = p
        while prime_power <= limit:
            prime_powers.append(p)
            powers_keys.append(prime_power)
            prime_power *= p
    
    return [{k: p for p, k in zip(prime_powers, powers_keys)}[k] for k in sorted(powers_keys)]

def pollard_p_1(n: int, pps: Optional[List[int]] = None, a: Optional[int] = None, k: Optional[int] = None) -> int:
    """ Pollard's p-1 algorithm to factorise a composite number n """
    k = k or 5
    a = a or 13
    b = k
    pps = pps or prime_powers_to(97)
    while True:
        m = product(pps[b-k:b])
        a = pow(a, m, n)
        g = gcd(a - 1, n)
        if 1 < g < n:
            break
        if g == n:
            raise Exception
        b += k
        if b > len(pps):
            break
    return g

In [ ]:
c = 500551

print("\nPollard p-1 method to find a factor")
g = pollard_p_1(c)
v = c // g
print(f"{g}.{v} = {g * v}")
print(ifactors(g-1))
print(ifactors(v-1))

pps = prime_powers_to(191)
print(f"This should give us the factor: {math.gcd((13 ** product(pps[:10]) % c) - 1, c)}")
#pps = [pp for pp in pps if pp != 13]
#print(f"This should give us the factor: {math.gcd((13 ** product(pps) % c) - 1, c)}")


In [ ]:
"""
pollard_p_1(100000000000000000000000000000001)
Pollard p-1 method to find a factor
(5, 30205667184746955447497956946313, 7, 1)
(10, 99692962616094161225683562481138, 2, 1)
(15, 69150440574603398719983360034106, 3, 1)
(20, 23455474226293589619844165171354, 41, 19841)
19841.5040068544932211078070661761 = 100000000000000000000000000000001
"""

__Question 4 (10 marks)__

Take the number $d$ assigned to you on page and use the $p-1$ method to find both factors of $d$. You may NOT use the maple procedure provided in weblearn.

In [ ]:
d = 29213
# print(ifactors(d))
# [(131, 1), (223, 1)]
print(ifactors(130))
print(ifactors(232))
print(pollard_p_1(d))

first_factor_pps = prime_powers_to(13)
print(first_factor_pps)
print(f"This should give us the 1st factor: {math.gcd((2 ** product(first_factor_pps) % d) - 1, d)}")

second_factor_pps = [pp for pp in prime_powers_to(37) if pp != 13]
print(second_factor_pps)
print(f"This should give us the 2nd factor: {math.gcd((2 ** product(second_factor_pps) % d) - 1, d)}")

__Question 5 (15 marks)__

Take the same number $d$ and now factorise using the Quadratic Sieve method. You may use Maple commands included in the week 10 workshop folder.

__Question 6 (15 marks)__

The Maple worksheet in Weblearn for this assignment shows part of an attempt to factorise $N = 9263 = 59*157$ using the Number Field Sieve. In this we claim the following:

&nbsp;&nbsp;&nbsp;&nbsp;$A = [0 , 1, 0]$ is a prime(irreducible) element of $\mathbb{Z}(\sqrt[3]{-2})$   with norm = 2
    
&nbsp;&nbsp;&nbsp;&nbsp;$B = [-1 ,- 1, 0]$ is a prime(irreducible) element of $\mathbb{Z}(\sqrt[3]{-2})$   with norm = 3
    
&nbsp;&nbsp;&nbsp;&nbsp;$C = [1, 0 , 1]$ is a prime(irreducible) element of $\mathbb{Z}(\sqrt[3]{-2})$   with norm = 5  
    
&nbsp;&nbsp;&nbsp;&nbsp;$D = [1 , 1, -1]$ is a prime(irreducible) element of $\mathbb{Z}(\sqrt[3]{-2})$   with norm = 11
    
&nbsp;&nbsp;&nbsp;&nbsp;$E = [1 , -2, 0]$ is a prime(irreducible) element of $\mathbb{Z}(\sqrt[3]{-2})$   with norm = 17
    
&nbsp;&nbsp;&nbsp;&nbsp;$F = [3 , 0, -1]$ is a prime(irreducible) element of $\mathbb{Z}(\sqrt[3]{-2})$  with norm = 23

We also derive a $48 x 15$ matrix $R$ consisting of values of $a$ and $b$ such that $a + 21b$ can be factorised using small primes and $[a, b, 0]$ can 
be factorised in Z(∛(-2) using just the primes ${A, B, C, D, E, F}$ and a unit element U = $[1, 1, 0]$.

&nbsp;&nbsp;&nbsp;&nbsp;Prove the statement *iii)* $C$ above allocated to you on page 1 using the definition of a norm.
    
&nbsp;&nbsp;&nbsp;&nbsp;Show that rows 23, 37, 41 and 45 of the matrix  form a linearly dependent set modulo 2
    
&nbsp;&nbsp;&nbsp;&nbsp;Hence find an equation of the form $u_2 – v_2$ such that $u_2 – v_2$ and $N$ have a common factor of $59$ thus 
factorising N. You may use the Maple procedure for multiplication in $\mathbb{Z}(\sqrt[3]{-2})$ to help you find $u$ and $v$.

__Q7__

Compare the methods for integer factorisation you have seen in the module and summarises their strengths and weaknesses, including the size of numbers that can be factorised, usability and whether or not they work for a broad range of numbers. 

1. Trial Division:

    Strengths: Simple and easy to implement. Works well for small numbers.
    Weaknesses: Inefficient for large numbers. Time complexity is $O(\sqrt(n))$.

1. Pollard's rho algorithm:

    Strengths: Efficient for semi-primes (numbers with two large prime factors).
    Weaknesses: May not work well for small prime factors or smooth numbers. Time complexity varies.

1. Quadratic Sieve (QS):

    Strengths: Efficient for numbers with small to medium-sized factors. Can handle larger numbers compared to basic methods.
    Weaknesses: Complex implementation. Not as efficient for very large numbers. Time complexity is sub-exponential.

1. General Number Field Sieve (GNFS):

    Strengths: Currently the most efficient algorithm for large numbers. Used for RSA key factorization.
    Weaknesses: Complex and resource-intensive. Practical for very large numbers only. Time complexity is sub-exponential.

1. Elliptic Curve Factorization (ECM):

    Strengths: Effective for numbers with small to medium-sized factors. Can be faster than QS in certain cases.
    Weaknesses: Implementation complexity. Not as efficient as GNFS for very large numbers.

1. Fermat's Factorization Method:

    Strengths: Simple and fast for numbers with certain forms. Can work well for numbers of the form a2−b2a2−b2.
    Weaknesses: Limited applicability. Not suitable for all types of numbers.

1. Lenstra Elliptic-Curve Factorization (LECF):

    Strengths: Effective for medium-sized numbers with small factors.
    Weaknesses: Less efficient for very large numbers compared to GNFS.

1. Trial Division with Wheel Factorization:

    Strengths: An improvement over basic trial division, reducing the number of unnecessary divisions.
    Weaknesses: Still impractical for very large numbers.

__Summary__:

Small Numbers: For small numbers, simple methods like trial division or Pollard's rho algorithm are effective and easy to implement.

Medium-sized Numbers: Quadratic Sieve, Elliptic Curve Factorization, and Lenstra Elliptic-Curve Factorization are good choices for numbers with small to medium-sized factors.

Large Numbers: General Number Field Sieve (GNFS) is the most powerful algorithm for very large numbers but is complex and resource-intensive. Elliptic Curve Factorization and Lenstra Elliptic-Curve Factorization may be practical for moderately large numbers.

Usability: Simple methods are more user-friendly but limited in their applicability. Advanced methods provide greater flexibility but require more expertise and resources.

Broad Range: Each method has its niche. Trial division and Pollard's rho algorithm work for a broad range but are less efficient for very large numbers. Advanced methods like GNFS and ECM are tailored for larger numbers.

Choosing the right factorization method depends on the size and nature of the number you are dealing with, as well as the available computational resources.